# S&P500 trend follwing on mt5 

idea for this research is to see what the best way would be to deploy algo that would time the market on us500.

1. Using bollinger bands 
2. Using emas 
3. Using engulfing candles or pinbar candles


### Result



In [8]:
'''
Start                     2024-01-02 05:45:00
End                       2024-03-28 23:45:00
Duration                     86 days 18:00:00
Exposure Time [%]                   73.943294
Equity Final [$]                      14888.2
Equity Peak [$]                       14963.6
Return [%]                             48.882
Buy & Hold Return [%]                9.992667
Return (Ann.) [%]                  390.245473
Volatility (Ann.) [%]               250.15073
Sharpe Ratio                         1.560041
Sortino Ratio                       21.004768
Calmar Ratio                        30.294429
Max. Drawdown [%]                  -12.881757
Avg. Drawdown [%]                   -0.690614
Max. Drawdown Duration       13 days 22:00:00
Avg. Drawdown Duration        0 days 14:26:00
# Trades                                   90
Win Rate [%]                        33.333333
Best Trade [%]                       1.172458
Worst Trade [%]                     -0.523848
Avg. Trade [%]                       0.087714
Max. Trade Duration           4 days 04:45:00
Avg. Trade Duration           0 days 17:03:00
Profit Factor                        1.607989
'''

'\nStart                     2023-09-01 02:25:00\nEnd                       2024-03-28 07:05:00\nDuration                    209 days 04:40:00\nExposure Time [%]                   95.447336\nEquity Final [$]                     13245.35\nEquity Peak [$]                      13306.55\nReturn [%]                            32.4535\nBuy & Hold Return [%]               16.431913\nReturn (Ann.) [%]                   61.128558\nVolatility (Ann.) [%]               40.268482\nSharpe Ratio                         1.518025\nSortino Ratio                         3.78781\nCalmar Ratio                         2.791753\nMax. Drawdown [%]                  -21.896118\nAvg. Drawdown [%]                   -0.404287\nMax. Drawdown Duration       80 days 06:05:00\nAvg. Drawdown Duration        0 days 16:47:00\n# Trades                                 1029\nWin Rate [%]                        32.361516\nBest Trade [%]                        1.31746\nWorst Trade [%]                     -0.600701\nAvg. Trade

In [9]:
import MetaTrader5 as mt5
import pandas as pd
import ta
import time
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from datetime import datetime
mt5.initialize()

True

In [151]:
symbol = 'US500'
timeframe = mt5.TIMEFRAME_M15
start_date = datetime(2023,1,1)
end_time = datetime.now()

def get_data(symbol, timeframe, start_date, end_time):
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_time)
    df = pd.DataFrame(rates)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df.set_index('time', inplace=True)
    df.columns = df.columns.str.capitalize()
    return df

us500 = get_data(symbol, timeframe, start_date, end_time)
us500.dropna(inplace=True)


In [152]:
def generate_signal(data):
    data['Lower_BB'] = ta.volatility.bollinger_lband(data['Close'], window=20)
    data['RSI'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()
    data['Signals'] = 0
    
    for i in range(1, len(data)):
        
        if (data['RSI'].iloc[i] <= 40 and data['Close'].iloc[i] > data['Lower_BB'].iloc[i]) and (data['Close'].iloc[i-1] <= data['Lower_BB'].iloc[i-1]):
            data.at[data.index[i], 'Signals'] = 1
        else:
            data.at[data.index[i], 'Signals'] = 0
        
    return data

us500 = generate_signal(us500)
us500.dropna(inplace=True)
us500


,Open,High,Low,Close,Tick_volume,Spread,Real_volume,Lower_BB,RSI,Signals
time,,,,,,,,,,
2023-01-03 05:45:00,3842.10,3845.10,3842.10,3844.10,190,50,0,3822.098805,54.227757,0
2023-01-03 06:00:00,3844.40,3844.90,3841.10,3842.10,191,50,0,3822.967648,50.808162,0
2023-01-03 06:15:00,3842.10,3843.40,3841.60,3843.40,151,50,0,3823.307678,52.887784,0
2023-01-03 06:30:00,3843.60,3845.00,3843.10,3843.10,135,50,0,3823.673081,52.337901,0
2023-01-03 06:45:00,3843.40,3843.60,3841.40,3841.60,171,50,0,3824.175410,49.563122,0
...,...,...,...,...,...,...,...,...,...,...
2024-03-28 22:45:00,5261.06,5262.81,5249.48,5251.48,767,20,0,5250.065578,43.232501,0
2024-03-28 23:00:00,5251.23,5253.73,5247.73,5252.23,417,36,0,5249.462428,44.368740,0
2024-03-28 23:15:00,5251.98,5253.48,5251.73,5251.98,98,36,0,5248.921493,44.052220,0


In [88]:


# Without RSI

def generate_signal(data):
    data['Lower_BB'] = ta.volatility.bollinger_lband(data['Close'], window=20)
    data['Signals'] = 0
    
    for i in range(1, len(data)):
        
        if (data['Close'].iloc[i] > data['Lower_BB'].iloc[i]) and (data['Close'].iloc[i-1] <= data['Lower_BB'].iloc[i-1]):
            data.at[data.index[i], 'Signals'] = 1
        else:
            data.at[data.index[i], 'Signals'] = 0
        
    return data

us500 = generate_signal(us500)
us500.dropna(inplace=True)
us500

,Open,High,Low,Close,Tick_volume,Spread,Real_volume,Lower_BB,Signals
time,,,,,,,,,
2024-01-02 17:35:00,4732.20,4732.50,4730.00,4732.20,141,20,0,4732.796549,0
2024-01-02 17:40:00,4732.00,4736.50,4731.70,4735.70,194,20,0,4732.370367,1
2024-01-02 17:45:00,4735.50,4737.50,4733.20,4733.50,165,20,0,4731.900443,0
2024-01-02 17:50:00,4733.70,4735.70,4733.00,4734.00,164,20,0,4731.448969,0
2024-01-02 17:55:00,4733.70,4734.70,4731.70,4733.70,129,20,0,4730.997339,0
...,...,...,...,...,...,...,...,...,...
2024-03-28 23:35:00,5251.48,5251.48,5249.23,5249.23,35,36,0,5246.243161,0
2024-03-28 23:40:00,5249.48,5249.98,5249.23,5249.98,35,36,0,5246.239595,0
2024-03-28 23:45:00,5249.98,5249.98,5249.23,5249.73,21,36,0,5246.225695,0


In [153]:
us500['Signals'].sum()


492

In [154]:
def SIGNAL():
    return us500.Signals

In [157]:


class BollingerBandBuy(Strategy):
    initsize = 5.0
    mysize = initsize
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        
    def next(self):
        super().next()
        if (self.signal1 > 0 and len(self.trades)==0 and len(self.closed_trades)>0 and self.closed_trades[-1].pl < 0) :
            self.mysize = self.mysize*2
        elif len(self.closed_trades)>0 and self.closed_trades[-1].pl > 0:
            self.mysize = self.initsize
            
        if self.signal1 == 1:
            sl1 = self.data.Close[-1] - 250e-1
            tp1 = self.data.Close[-1] + 550e-1
            self.buy(sl = sl1, tp = tp1, size=self.mysize)      
            
bt = Backtest(us500, BollingerBandBuy, cash=10000, exclusive_orders=True, margin= 1/100)

output = bt.run()
output  


Start                     2023-01-03 05:45:00
End                       2024-03-28 23:45:00
Duration                    450 days 18:00:00
Exposure Time [%]                    71.02059
Equity Final [$]                      20574.2
Equity Peak [$]                       20649.6
Return [%]                            105.742
Buy & Hold Return [%]               36.565906
Return (Ann.) [%]                   76.728506
Volatility (Ann.) [%]               84.705071
Sharpe Ratio                         0.905831
Sortino Ratio                        4.136924
Calmar Ratio                         2.243432
Max. Drawdown [%]                  -34.201389
Avg. Drawdown [%]                   -0.861851
Max. Drawdown Duration      189 days 03:30:00
Avg. Drawdown Duration        1 days 15:40:00
# Trades                                  492
Win Rate [%]                        30.894309
Best Trade [%]                       1.447186
Worst Trade [%]                     -0.663185
Avg. Trade [%]                    

In [150]:
bt.plot()

Row(id='15643', ...)